In [2]:
import numpy as np
from scipy.special import loggamma
normalize = lambda x: x/x.sum()
logfactorial = lambda x: loggamma(x+1)


class prob_vector:
    def __init__(self,array=None,components=20,beads=10000):
        if array is None:
            self.counts = np.ones(components,dtype=int)*(beads//components)
            self.counts[:beads%components]+=1
        elif array.dtype == int:
            self.counts = array
        elif np.isclose(1.,np.sum(array)):
            self.counts = np.floor(array*beads).astype(int)
            self.counts[np.argsort(array)[:beads%self.counts.sum()]]+=1

        self.prob = normalize(self.counts)

    def proposal(self,rate=.01):
        movables = np.random.binomial(self.counts,2*rate)
        new_counts = self.counts - movables 

        mvleft = np.random.binomial(movables,.5)
        mvright = movables - mvleft 

        new_counts[:-1] += mvleft[1:]
        new_counts[0] += mvleft[0] #the ones selected to move left from 0 stay in place

        new_counts[1:] += mvright[:-1]
        new_counts[-1] += mvright[-1] #the ones selected to move left from -1 stay in place

        ##print(movables.sum(),np.abs(self.counts-new_counts).sum())
        return prob_vector(new_counts,new_counts.size,new_counts.sum())
    
    def multinomial_logprob(self,alpha):
        beads = self.counts.sum()
        p = self.alpha/self.alpha.sum()

        log_prefactor = logfactorial(beads) - logfactorial(self.counts).sum()
        return log_prefactor + (self.counts*np.log(p)).sum()
        
